In [1]:
#

In [2]:
import torch
import torch.nn as nn
import torch.optim as optim
import torch.nn.functional as F

In [7]:
def discritized_mix_logistic_loss(x, l):
    '''
    x -> target BHWC
    l -> output BHWC
    '''
    
    l_shape = l.shape
    nr_mix = int(l_shape[-1]/10)
    
    split = [nr_mix, 3 * nr_mix, 3 * nr_mix, 3 * nr_mix]
    pi_i, mu_i, log_s_i, rgb_coefficients = torch.split(l, split_size_or_sections = split, dim = -1)
    
    log_s_i = torch.clamp(log_s_i, -7.)
    log_s_i = torch.cat(torch.split(log_s_i.unsqueeze(-2), 3, -1), -2)
    rgb_coefficients = torch.nn.tanh(rgb_coefficients)
    one_over_s_i = torch.exp(-log_s_i)
    
    mu_r, mu_g, mu_b = torch.split(mu_i, split_size_or_sections = 3, dim = -1)
    c0, c1, c2 = torch.split(rgb_coefficients, split_size_or_sections = 3,dim = -1)
    x_r, x_g, x_b = torch.split(x, split_size_or_sections = 3, axis = -1)
    
    mu_g += c0 * x_r
    mu_b += c1 * x_r + c2 * x_g
    
    mu_i = torch.cat([mu_r.unsqueeze(-2),
                     mu_g.unsqueeze(-2),
                     mu_b.unsqueeze(-2)], axis = -2)
    
    x = x.unsqueeze(-1)
    x_minus_mu = torch.subtract(x, mu_i)
    
    plus_in = one_over_s_i * (x_minus_mu + 1. / 255.)
    min_in = one_over_s_i * (x_minus_mu + 1. / 255.)

    cdf_plus = torch.sigmoid(plus_in) # CDF of logistics at x + 0.5
    cdf_min = torch.sigmoid(min_in) # CDF of logistics at x - 0.5

    cdf_delta = cdf_plus - cdf_min # probability of x in bin [x - 0.5, x + 0.5]
    log_cdf_delta = torch.log(torch.clamp(cdf_delta, min=1e-12))
    log_cdf_plus = torch.log(torch.clamp(torch.sigmoid(one_over_s_i * (0.5 - means)),
                                                           min=1e-12))
    log_cdf_min = torch.log(torch.clamp(1 - torch.sigmoid(one_over_s_i * (d - 1.5 - means)),
                                            min=1e-12))

    x_log_probs = torch.where(x < 0.001, log_cdf_plus,
                                  torch.where(x > d - 1 - 1e-3,
                                              log_cdf_min, log_cdf_delta))
    pi_log_probs = F.log_softmax(pi_i - torch.max(pi_i, -1, keepdims =True)).unsqueeze(0)
    log_probs = x_log_probs + pi_log_probs
    return torch.logsumexp(log_probs, dim=1)

tensor([10.])

In [11]:
l = torch.randn(10,28,28,100)

In [13]:
l_shape = l.shape
nr_mix = int(l_shape[-1]/10)
    
split = [nr_mix, 3 * nr_mix, 3 * nr_mix, 3 * nr_mix]
pi_i, mu_i, log_s_i, rgb_coefficients = torch.split(l, split_size_or_sections = split, dim = -1)
    
log_s_i = torch.clamp(log_s_i, -7.)


In [30]:
[int(i) for i in log_s_i.shape] + [10 * 3]

[10, 28, 28, 30, 30]

In [26]:
out = torch.cat(torch.split(log_s_i.unsqueeze(-2), 3, -1), -2)

In [27]:
out.shape

torch.Size([10, 28, 28, 10, 3])